In [22]:
%matplotlib inline
import numpy as np
from bokeh.layouts import row, column, gridplot
from bokeh.models import ColumnDataSource, Slider, Select
from bokeh.plotting import curdoc, figure, show, output_file
from bokeh.driving import count
from bokeh.palettes import Spectral11
import itertools

import json
from os.path import dirname, join
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import collections

BUFSIZE = 200
filename = '17700.json'
    

In [35]:
def jsonToDF(filename):
    initialize = True
    json1_file = open(filename)
    json1_str = json1_file.read()
    json1_data = json.loads(json1_str)
    for link in json1_data.keys():
        tsdict = json1_data[link]
        keys = sorted(list(map(int, tsdict.keys())))
        sorted_val = []
        for k in keys:
            sorted_val.append(tsdict[str(k)])
        if initialize:
            df = pd.DataFrame({'time': keys, link: sorted_val})
            df.set_index('time')
            initialize = False
        else:
            df = pd.concat([df, pd.DataFrame({link: sorted_val})], axis=1)
    return df

df = jsonToDF(filename)
print(df.head())
#df.plot(figsize=(17,20))
dicts = df.to_dict(orient='records')

            time  pppl-rt5  srs-rt3  eqx-ash-asw1  orau-rt3    pnwg-ssw1  \
0  1529280000000       0.0      0.0           0.0       0.0  2563.698814   
1  1529280030000       0.0      0.0           0.0       0.0  3289.142940   
2  1529280060000       0.0      0.0           0.0       0.0  2440.466643   
3  1529280090000       0.0      0.0           0.0       0.0  3251.836702   
4  1529280120000       0.0      0.0           0.0       0.0  3830.225718   

   lbl-2002-asw1  wash-ar1    wash-cr5  osti-rt2     ...          aofa-cr5  \
0            0.0       0.0  289.479020       0.0     ...       2999.733333   
1            0.0       0.0  357.782284       0.0     ...       2601.866667   
2            0.0       0.0  327.292855       0.0     ...       2343.200000   
3            0.0       0.0  251.111613       0.0     ...       2378.666667   
4            0.0       0.0  268.645056       0.0     ...       2745.066667   

   cern-513-cr5  bnl-lsw4     bnl-lsw2       bnl-lsw1      bost-cr5  \
0  

In [36]:
source = ColumnDataSource(df)
links = list(df.columns.values)
for dict in dicts:
    for k in links:
        dict[k] = [dict[k]]
links.remove('time')

p = figure(plot_height=500, tools="xpan,xwheel_zoom,xbox_zoom,reset", x_axis_type=None, y_axis_location="right")
p.x_range.follow = "end"
p.x_range.follow_interval = 100
p.x_range.range_padding = 0
ts = Select(value="all", options=["all"] + links)

numlines=len(links)
palette=itertools.cycle(Spectral11[0:numlines])

for i in range(numlines):
    p.line(x='time', y=links[i], alpha=0.2, line_width=3, color=next(palette), source=source)


{'time': [1529280000000.0], 'pppl-rt5': [0.0], 'srs-rt3': [0.0], 'eqx-ash-asw1': [0.0], 'orau-rt3': [0.0], 'pnwg-ssw1': [2563.698813597361], 'lbl-2002-asw1': [0.0], 'wash-ar1': [0.0], 'wash-cr5': [289.47902018229166], 'osti-rt2': [0.0], 'netl-mgn-rt1': [19843.94553527832], 'star-cr5': [150.25999806722004], 'albq-asw1': [0.0], 'elpa-cr5': [868248365.3333334], 'jgi-asw1': [0.0], 'sunn-asw1': [0.0], 'netl-mgn-ssw1': [11813.002880350748], 'nash-cr5': [13630160.4], 'albq-cr5': [300147198.73333335], 'anl-ssw1': [5369.891113408406], 'sunn-cr5': [34230.96666666667], 'denv-cr5': [1832587237.3333333], 'osti-ssw1': [1389.9321883201599], 'llnl-mr2': [0.0], 'newy-asw1': [284082.02643229166], 'star-asw1': [145.2618887980779], 'nersc-2960-1': [3879.205244064331], 'sdsc-2940-1': [2452.3480570475263], 'snll-mr2': [68.67121857802073], 'chic-2960-1': [3021.6771589279174], 'chic-asw1': [372329.7859049479], 'esnet-lsw4': [42176.03204447428], 'denv-2960-1': [3584.4166208902993], 'slac-2940-1': [2291.8389973

In [29]:
@count()
def update(t):
    global dicts
    new_data = dicts[t % len(dicts)]

#     if   ts.value == MA12:  new_data['ma'] = [ma12]
#     elif ts.value == MA26:  new_data['ma'] = [ma26]
#     elif ts.value == EMA12: new_data['ma'] = [ema12]
#     elif ts.value == EMA26: new_data['ma'] = [ema26]

    source.stream(new_data, 300)

curdoc().add_root(column(row(ts), gridplot([[p]], toolbar_location="left", plot_width=1000)))
curdoc().add_periodic_callback(update, 50)
curdoc().title = "SNMP"

In [14]:
import datetime
your_timestamp = 1331856000000
date = datetime.datetime.fromtimestamp(1529280000000 / 1e3)
str(date)

'2018-06-17 17:00:00'

In [1]:
import numpy as np
np.random.seed(1)

from bokeh.layouts import row, column, gridplot
from bokeh.models import ColumnDataSource, Slider, Select
from bokeh.plotting import curdoc, figure
from bokeh.driving import count

BUFSIZE = 200
MA12, MA26, EMA12, EMA26 = '12-tick Moving Avg', '26-tick Moving Avg', '12-tick EMA', '26-tick EMA'

source = ColumnDataSource(dict(
    time=[], average=[], low=[], high=[], open=[], close=[],
    ma=[], macd=[], macd9=[], macdh=[], color=[]
))

p = figure(plot_height=500, tools="xpan,xwheel_zoom,xbox_zoom,reset", x_axis_type=None, y_axis_location="right")
p.x_range.follow = "end"
p.x_range.follow_interval = 100
p.x_range.range_padding = 0

p.line(x='time', y='average', alpha=0.2, line_width=3, color='navy', source=source)
p.line(x='time', y='ma', alpha=0.8, line_width=2, color='orange', source=source)
p.segment(x0='time', y0='low', x1='time', y1='high', line_width=2, color='black', source=source)
p.segment(x0='time', y0='open', x1='time', y1='close', line_width=8, color='color', source=source)

p2 = figure(plot_height=250, x_range=p.x_range, tools="xpan,xwheel_zoom,xbox_zoom,reset", y_axis_location="right")
p2.line(x='time', y='macd', color='red', source=source)
p2.line(x='time', y='macd9', color='blue', source=source)
p2.segment(x0='time', y0=0, x1='time', y1='macdh', line_width=6, color='black', alpha=0.5, source=source)

mean = Slider(title="mean", value=0, start=-0.01, end=0.01, step=0.001)
stddev = Slider(title="stddev", value=0.04, start=0.01, end=0.1, step=0.01)
mavg = Select(value=MA12, options=[MA12, MA26, EMA12, EMA26])

def _create_prices(t):
    last_average = 100 if t==0 else source.data['average'][-1]
    returns = np.asarray(np.random.lognormal(mean.value, stddev.value, 1))
    average =  last_average * np.cumprod(returns)
    high = average * np.exp(abs(np.random.gamma(1, 0.03, size=1)))
    low = average / np.exp(abs(np.random.gamma(1, 0.03, size=1)))
    delta = high - low
    open = low + delta * np.random.uniform(0.05, 0.95, size=1)
    close = low + delta * np.random.uniform(0.05, 0.95, size=1)
    return open[0], high[0], low[0], close[0], average[0]

def _moving_avg(prices, days=10):
    if len(prices) < days: return [100]
    return np.convolve(prices[-days:], np.ones(days, dtype=float), mode="valid") / days

def _ema(prices, days=10):
    if len(prices) < days or days < 2: return [prices[-1]]
    a = 2.0 / (days+1)
    kernel = np.ones(days, dtype=float)
    kernel[1:] = 1 - a
    kernel = a * np.cumprod(kernel)
    # The 0.8647 normalizes out that we stop the EMA after a finite number of terms
    return np.convolve(prices[-days:], kernel, mode="valid") / (0.8647)

@count()
def update(t):
    open, high, low, close, average = _create_prices(t)
    color = "green" if open < close else "red"

    new_data = dict(
        time=[t],
        open=[open],
        high=[high],
        low=[low],
        close=[close],
        average=[average],
        color=[color],
    )

    close = source.data['close'] + [close]
    ma12 = _moving_avg(close[-12:], 12)[0]
    ma26 = _moving_avg(close[-26:], 26)[0]
    ema12 = _ema(close[-12:], 12)[0]
    ema26 = _ema(close[-26:], 26)[0]

    if   mavg.value == MA12:  new_data['ma'] = [ma12]
    elif mavg.value == MA26:  new_data['ma'] = [ma26]
    elif mavg.value == EMA12: new_data['ma'] = [ema12]
    elif mavg.value == EMA26: new_data['ma'] = [ema26]

    macd = ema12 - ema26
    new_data['macd'] = [macd]

    macd_series = source.data['macd'] + [macd]
    macd9 = _ema(macd_series[-26:], 9)[0]
    new_data['macd9'] = [macd9]
    new_data['macdh'] = [macd - macd9]

    source.stream(new_data, 300)

curdoc().add_root(column(row(mean, stddev, mavg), gridplot([[p], [p2]], toolbar_location="left", plot_width=1000)))
curdoc().add_periodic_callback(update, 50)
curdoc().title = "OHLC"

In [4]:
_create_prices(0)

(105.77556002562126,
 106.7134742510443,
 105.56676727306952,
 105.71939940183024,
 106.71310807070746)